In [1]:
import os
os.chdir("../")
from datetime import timedelta
from aladdin import PostgreSQLConfig, RedshiftSQLConfig, RedisConfig, FeatureView, CombinedFeatureView, FeatureViewMetadata, CombinedFeatureViewMetadata, Entity, String, Bool, UUID, Int32, Int64, Float, Double, CreatedAtTimestamp, Ratio, Contains, Equals, EventTimestamp, DateComponent, TimeDifferance, DifferanceBetween, FileSource, FeatureStore
from aladdin.model import ModelService
from aladdin.split_strategy import UniformSplitStrategy
import numpy as np

In [2]:
os.chdir("example")
store = FeatureStore.experimental()

In [3]:
class Match(FeatureView):

    metadata = FeatureViewMetadata(
        name="match",
        description="Features about football matches",
        tags={},
        batch_source=FileSource(
            path="https://raw.githubusercontent.com/footballcsv/cache.footballdata/master/2020-21/eng.1.csv",
            mapping_keys={
                "HT": "half_time_score",
                "FT": "full_time_score",
                "Team 2": "team_2",
                "Team 1": "team_1",
                "Date": "date",
            }
        )
    )

    team_1 = Entity(String())
    team_2 = Entity(String())
    date = EventTimestamp(max_join_with=timedelta(days=365))

    full_time_score = String().description("the scores at full time")
    half_time_score = String().description("the scores at half time")

    is_liverpool = (team_1 == "Liverpool").description("If the home team is Liverpool")

    score_array = half_time_score.split("-")

    half_time_team_1_score = score_array.transformed(lambda df: df["score_array"].str[0].replace({np.nan: 0}).astype(int))
    half_time_team_2_score = score_array.transformed(lambda df: df["score_array"].str[1].replace({np.nan: 0}).astype(int))

    half_time_differance = half_time_team_1_score - half_time_team_2_score
    half_time_sum = half_time_team_1_score + half_time_team_2_score


store.add_feature_view(Match())

df = await store.feature_view("match").all().to_df()

In [4]:
df

,date,team_1,full_time_score,half_time_score,team_2,score_array,is_liverpool,half_time_team_2_score,half_time_team_1_score,half_time_differance,half_time_sum
0,2020-09-12 00:00:00+00:00,Fulham,0-3,0-1,Arsenal,"[0, 1]",False,1,0,-1,1
1,2020-09-12 00:00:00+00:00,Crystal Palace,1-0,1-0,Southampton,"[1, 0]",False,0,1,1,1
2,2020-09-12 00:00:00+00:00,Liverpool,4-3,3-2,Leeds,"[3, 2]",True,2,3,1,5
3,2020-09-12 00:00:00+00:00,West Ham,0-2,0-0,Newcastle,"[0, 0]",False,0,0,0,0
4,2020-09-13 00:00:00+00:00,West Brom,0-3,0-0,Leicester,"[0, 0]",False,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
180,2021-01-19 00:00:00+00:00,Leicester,2-0,2-0,Chelsea,"[2, 0]",False,0,2,2,2
181,2021-01-20 00:00:00+00:00,Man City,2-0,0-0,Aston Villa,"[0, 0]",False,0,0,0,0
182,2021-01-20 00:00:00+00:00,Fulham,1-2,1-1,Man United,"[1, 1]",False,1,1,0,2
183,2021-01-21 00:00:00+00:00,Liverpool,0-1,0-0,Burnley,"[0, 0]",True,0,0,0,0


In [5]:
from datetime import datetime


await store.features_for({
    "team_1": ["Man City", "Man City"],
    "team_2": ["Arsenal", "Liverpool"],
    "event_timestamp": [datetime.utcnow(), datetime.utcnow()]
},
features=[
    "match:date",
    "match:full_time_score",
    "match:score_array",
    "match:half_time_team_1_score",
]
).to_df()

,score_array,half_time_team_1_score,event_timestamp,team_2,team_1,date,full_time_score
0,"[1, 0]",1,2022-06-27 09:20:49.434895,Arsenal,Man City,2020-10-17 00:00:00+00:00,1-0
1,"[1, 1]",1,2022-06-27 09:20:49.434898,Liverpool,Man City,2020-11-08 00:00:00+00:00,1-1


In [6]:
await store.feature_view("match").all(limit=20).to_df()

,date,team_1,full_time_score,half_time_score,team_2,score_array,is_liverpool,half_time_team_2_score,half_time_team_1_score,half_time_differance,half_time_sum
0,2020-09-12 00:00:00+00:00,Fulham,0-3,0-1,Arsenal,"[0, 1]",False,1,0,-1,1
1,2020-09-12 00:00:00+00:00,Crystal Palace,1-0,1-0,Southampton,"[1, 0]",False,0,1,1,1
2,2020-09-12 00:00:00+00:00,Liverpool,4-3,3-2,Leeds,"[3, 2]",True,2,3,1,5
3,2020-09-12 00:00:00+00:00,West Ham,0-2,0-0,Newcastle,"[0, 0]",False,0,0,0,0
4,2020-09-13 00:00:00+00:00,West Brom,0-3,0-0,Leicester,"[0, 0]",False,0,0,0,0
5,2020-09-13 00:00:00+00:00,Tottenham,0-1,0-0,Everton,"[0, 0]",False,0,0,0,0
6,2020-09-14 00:00:00+00:00,Brighton,1-3,0-1,Chelsea,"[0, 1]",False,1,0,-1,1
7,2020-09-14 00:00:00+00:00,Sheffield United,0-2,0-2,Wolves,"[0, 2]",False,2,0,-2,2
8,2020-09-19 00:00:00+00:00,Everton,5-2,2-1,West Brom,"[2, 1]",False,1,2,1,3
9,2020-09-19 00:00:00+00:00,Leeds,4-3,2-1,Fulham,"[2, 1]",False,1,2,1,3


In [7]:
async def some_feature_transformer(df):
    psql: PostgreSQLConfig = PostgreSQLConfig.localhost()
    enricher = psql.data_enricher("SELECT * FROM football.eng.5")
    data = await enricher.load()
    return data["score"].mean() - df["team_1_score"]


class OtherMatches(FeatureView):
    metadata = FeatureViewMetadata(
        name="other_matches",
        description="Features about football matches",
        tags={},
        batch_source=FileSource(
            path="https://raw.githubusercontent.com/footballcsv/england/master/2010s/2019-20/eng.1.csv",
            mapping_keys={
                "FT": "full_time_score",
                "Team 2": "team_2",
                "Team 1": "team_1",
                "Date": "date",
                "Round": "round"
            }
        )
    )

    team_1 = Entity(String())
    team_2 = Entity(String())
    date = EventTimestamp(max_join_with=timedelta(days=365))
    
    round = Int32()

    full_time_score = String().description("the scores at full time")

    score_array = full_time_score.split("-")

    team_1_score = score_array.transformed(lambda df: df["score_array"].str[0].replace({np.nan: 0}).astype(int))
    team_2_score = score_array.transformed(lambda df: df["score_array"].str[1].replace({np.nan: 0}).astype(int))

    differance = team_1_score - team_2_score


    some_feature = team_1_score.transformed(some_feature_transformer)



store.add_feature_view(OtherMatches())

df = await store.features_for({
    "team_1": ["Crystal Palace FC"],
    "team_2": ["Everton FC"],
    "event_timestamp": [datetime.utcnow()]
}, features=[
    # "match:half_time_team_1_score",
    # "match:is_liverpool",

    "other_matches:score_array",
    "other_matches:differance",
    "other_matches:team_1_score",
    "other_matches:team_2_score",
]).to_df()

In [8]:
df

,score_array,event_timestamp,team_1_score,team_1,team_2_score,differance,team_2
0,"[0, 0]",2022-06-27 09:20:50.310962,0,Crystal Palace FC,0,0,Everton FC


In [9]:
test_model = ModelService(
    features=[
        Match.select_all(),
        OtherMatches.select(lambda view: [
            view.team_1_score,
            view.team_2_score
        ]),
    ],
    name="test_model"
)
store.add_model_service(test_model)

In [10]:
await store.model("test_model").features_for({
    "team_1": ["Man City", "Leeds"],
    "team_2": ["Liverpool", "Arsenal"],
    "event_timestamp": [datetime.utcnow(), datetime.utcnow()]
}).to_df()

,score_array,score_array,is_liverpool,date,team_1_score,half_time_score,half_time_sum,half_time_team_2_score,half_time_team_1_score,event_timestamp,event_timestamp,full_time_score,full_time_score,team_1,team_1,team_2_score,half_time_differance,team_2,team_2
0,NaN,"[1, 0]",False,2020-10-17 00:00:00+00:00,0,1-0,1,0,1,2022-06-27 09:20:51.009161,2022-06-27 09:20:51.009161,NaN,1-0,Man City,Man City,0,1,Liverpool,Arsenal
1,NaN,"[1, 1]",False,2020-11-08 00:00:00+00:00,0,1-1,2,1,1,2022-06-27 09:20:51.009165,2022-06-27 09:20:51.009165,NaN,1-1,Leeds,Man City,0,0,Arsenal,Liverpool


In [11]:
await store.feature_view("match").previous(days=600).to_df()

,date,team_1,full_time_score,half_time_score,team_2,score_array,is_liverpool,half_time_team_2_score,half_time_team_1_score,half_time_differance,half_time_sum
68,2020-11-06 00:00:00+00:00,Brighton,0-0,0-0,Burnley,"[0, 0]",False,0,0,0,0
69,2020-11-06 00:00:00+00:00,Southampton,2-0,1-0,Newcastle,"[1, 0]",False,0,1,1,1
70,2020-11-07 00:00:00+00:00,Everton,1-3,1-2,Man United,"[1, 2]",False,2,1,-1,3
71,2020-11-07 00:00:00+00:00,Crystal Palace,4-1,3-1,Leeds,"[3, 1]",False,1,3,2,4
72,2020-11-07 00:00:00+00:00,Chelsea,4-1,2-1,Sheffield United,"[2, 1]",False,1,2,1,3
...,...,...,...,...,...,...,...,...,...,...,...
180,2021-01-19 00:00:00+00:00,Leicester,2-0,2-0,Chelsea,"[2, 0]",False,0,2,2,2
181,2021-01-20 00:00:00+00:00,Man City,2-0,0-0,Aston Villa,"[0, 0]",False,0,0,0,0
182,2021-01-20 00:00:00+00:00,Fulham,1-2,1-1,Man United,"[1, 1]",False,1,1,0,2
183,2021-01-21 00:00:00+00:00,Liverpool,0-1,0-0,Burnley,"[0, 0]",True,0,0,0,0


In [13]:
split_set = await store.feature_view("match").previous(days=600).split_with(
    UniformSplitStrategy(0.7, 0.2), 
    target_column="half_time_differance"
).use_pandas()

/Users/matsmollestad/Desktop/PersonalProj/aladdin-2.0/aladdin/split_strategy.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train = train.append(split(sub_group, 0, self.train_size_percentage))
/Users/matsmollestad/Desktop/PersonalProj/aladdin-2.0/aladdin/split_strategy.py:55: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test = test.append(
/Users/matsmollestad/Desktop/PersonalProj/aladdin-2.0/aladdin/split_strategy.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  develop = develop.append(
/Users/matsmollestad/Desktop/PersonalProj/aladdin-2.0/aladdin/split_strategy.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train = 

In [15]:
split_set.train_input

,date,team_1,full_time_score,half_time_score,team_2,score_array,is_liverpool,half_time_team_2_score,half_time_team_1_score,half_time_sum
68,2020-11-06 00:00:00+00:00,Brighton,0-0,0-0,Burnley,"[0, 0]",False,0,0,0
73,2020-11-07 00:00:00+00:00,West Ham,1-0,0-0,Fulham,"[0, 0]",False,0,0,0
74,2020-11-08 00:00:00+00:00,West Brom,0-1,0-0,Tottenham,"[0, 0]",False,0,0,0
76,2020-11-08 00:00:00+00:00,Man City,1-1,1-1,Liverpool,"[1, 1]",False,1,1,2
81,2020-11-21 00:00:00+00:00,Man United,1-0,0-0,West Brom,"[0, 0]",False,0,0,0
...,...,...,...,...,...,...,...,...,...,...
96,2020-11-30 00:00:00+00:00,Leicester,1-2,0-2,Fulham,"[0, 2]",False,2,0,2
159,2021-01-02 00:00:00+00:00,Brighton,3-3,1-3,Wolves,"[1, 3]",False,3,1,4
90,2020-11-28 00:00:00+00:00,Man City,5-0,3-0,Burnley,"[3, 0]",False,0,3,3
116,2020-12-13 00:00:00+00:00,Leicester,3-0,3-0,Brighton,"[3, 0]",False,0,3,3


In [16]:
split_set.test_input

,date,team_1,full_time_score,half_time_score,team_2,score_array,is_liverpool,half_time_team_2_score,half_time_team_1_score,half_time_sum
151,2020-12-29 00:00:00+00:00,Southampton,0-0,0-0,West Ham,"[0, 0]",False,0,0,0
153,2020-12-29 00:00:00+00:00,Man United,1-0,0-0,Wolves,"[0, 0]",False,0,0,0
154,2020-12-30 00:00:00+00:00,Newcastle,0-0,0-0,Liverpool,"[0, 0]",False,0,0,0
155,2021-01-01 00:00:00+00:00,Everton,0-1,0-0,West Ham,"[0, 0]",False,0,0,0
161,2021-01-03 00:00:00+00:00,Newcastle,1-2,0-0,Leicester,"[0, 0]",False,0,0,0
164,2021-01-12 00:00:00+00:00,Sheffield United,1-0,0-0,Newcastle,"[0, 0]",False,0,0,0
165,2021-01-12 00:00:00+00:00,Burnley,0-1,0-0,Man United,"[0, 0]",False,0,0,0
166,2021-01-12 00:00:00+00:00,Wolves,1-2,1-1,Everton,"[1, 1]",False,1,1,2
169,2021-01-14 00:00:00+00:00,Arsenal,0-0,0-0,Crystal Palace,"[0, 0]",False,0,0,0
173,2021-01-16 00:00:00+00:00,Fulham,0-1,0-0,Chelsea,"[0, 0]",False,0,0,0


In [22]:
split_set.develop_input

,date,team_1,full_time_score,half_time_score,team_2,score_array,is_liverpool,half_time_team_2_score,half_time_team_1_score,half_time_sum
176,2021-01-17 00:00:00+00:00,Liverpool,0-0,0-0,Man United,"[0, 0]",True,0,0,0
178,2021-01-18 00:00:00+00:00,Arsenal,3-0,0-0,Newcastle,"[0, 0]",False,0,0,0
181,2021-01-20 00:00:00+00:00,Man City,2-0,0-0,Aston Villa,"[0, 0]",False,0,0,0
182,2021-01-20 00:00:00+00:00,Fulham,1-2,1-1,Man United,"[1, 1]",False,1,1,2
183,2021-01-21 00:00:00+00:00,Liverpool,0-1,0-0,Burnley,"[0, 0]",True,0,0,0
172,2021-01-16 00:00:00+00:00,West Ham,1-0,1-0,Burnley,"[1, 0]",False,0,1,1
174,2021-01-16 00:00:00+00:00,Leicester,2-0,1-0,Southampton,"[1, 0]",False,0,1,1
177,2021-01-17 00:00:00+00:00,Man City,4-0,1-0,Crystal Palace,"[1, 0]",False,0,1,1
179,2021-01-19 00:00:00+00:00,West Ham,2-1,1-0,West Brom,"[1, 0]",False,0,1,1
146,2020-12-27 00:00:00+00:00,Wolves,1-1,0-1,Tottenham,"[0, 1]",False,1,0,1
